# Multimodal Analytics with Cortex AISQL

This notebook demonstrates:
- **AI_COMPLETE** with images
- **AI_TRANSCRIBE** for audio files
- **AI_PARSE_DOCUMENT** for OCR
- **TO_FILE** for file references


In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.sql("USE DATABASE AISQL_DB").collect()
session.sql("USE SCHEMA AISQL_SCHEMA").collect()
session.sql("USE WAREHOUSE AISQL_WH").collect()


## 1. AI_COMPLETE with Images

Analyze screenshots and images using vision models


In [ ]:
# Analyze images with AI_COMPLETE
sql = """
SELECT 
    relative_path,
    AI_COMPLETE('pixtral-large', 
        'Describe this image and identify any issues or errors visible.', 
        img_file) as image_analysis
FROM images
WHERE relative_path LIKE 'screenshot%'
LIMIT 5
"""

df_images = session.sql(sql).to_pandas()
st.subheader("Image Analysis Results")
st.dataframe(df_images)


## 2. AI_TRANSCRIBE for Audio

Transcribe voicemail audio files


In [ ]:
# Transcribe audio files
sql = """
SELECT 
    relative_path,
    AI_TRANSCRIBE(audio_file)['text'] as transcription,
    AI_SENTIMENT(AI_TRANSCRIBE(audio_file)['text']) as sentiment_score
FROM voicemails
LIMIT 10
"""

df_audio = session.sql(sql).to_pandas()
st.subheader("Audio Transcriptions")
st.dataframe(df_audio)

# Sentiment distribution
if len(df_audio) > 0:
    avg_sentiment = df_audio['SENTIMENT_SCORE'].mean()
    st.metric("Average Voicemail Sentiment", f"{avg_sentiment:.3f}")


## 3. AI_PARSE_DOCUMENT for OCR

Extract text from images using OCR


In [ ]:
# Parse documents with OCR
sql = """
SELECT 
    relative_path,
    AI_PARSE_DOCUMENT(img_file, {'mode': 'OCR'}) as extracted_text,
    LENGTH(AI_PARSE_DOCUMENT(img_file, {'mode': 'OCR'})) as text_length
FROM images
WHERE relative_path LIKE 'screenshot%'
LIMIT 5
"""

df_ocr = session.sql(sql).to_pandas()
st.subheader("OCR Extracted Text")

for idx, row in df_ocr.iterrows():
    with st.expander(f"{row['RELATIVE_PATH']} ({row['TEXT_LENGTH']} chars)"):
        st.write(row['EXTRACTED_TEXT'])
